In [ ]:
import geopandas as gpd
import pandas as pd

# Load the shapefile of Canada
canada_shapefile = gpd.read_file('./data/ler_000b16a_e.shp')

# Load the job outlook data
job_outlook_data = pd.read_excel('./data/20242026_outlook_n21_en_250117.xlsx')

canada_shapefile['ERUID'] = canada_shapefile['ERUID'].astype(int)

# Merge the data on the region code
merged_data = canada_shapefile.merge(job_outlook_data, left_on='ERUID', right_on='Economic Region Code')

import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Page 1: Geographical Map
page_1_layout = html.Div([
    html.H1("Job Outlook in Canada"),
    dcc.Graph(id='canada-map')
])

# Page 2: Specific Job Outlook Plot
page_2_layout = html.Div([
    html.H1("Specific Job Outlook Across Regions"),
    dcc.Dropdown(
        id='job-dropdown',
        options=[{'label': job, 'value': job} for job in merged_data['NOC Title'].unique()],
        value=merged_data['NOC Title'].unique()[0]
    ),
    dcc.Graph(id='job-outlook-plot')
])

# Update the page content based on the URL
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return page_1_layout

# Callback for the geographical map
@app.callback(Output('canada-map', 'figure'),
              [Input('url', 'pathname')])
def update_map(pathname):
    fig = px.choropleth(merged_data,
                        geojson=merged_data.geometry,
                        locations=merged_data.index,
                        color='Outlook',
                        hover_name='Economic Region Name',
                        projection="mercator")
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig

# Callback for the specific job outlook plot
@app.callback(Output('job-outlook-plot', 'figure'),
              [Input('job-dropdown', 'value')])
def update_job_outlook_plot(selected_job):
    filtered_data = merged_data[merged_data['NOC Title'] == selected_job]
    fig = px.bar(filtered_data,
                 x='Economic Region Name',
                 y='Outlook',
                 color='Outlook',
                 labels={'Outlook': 'Job Outlook'},
                 title=f'Job Outlook for {selected_job} Across Regions')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)